# 05 · Train Auto-K Cardinality Model

## Purpose

Learn to predict how many controls (1–3) to emit per artifact using calibrated scores.

## Inputs

- `data/processed/pairs/train.jsonl` and `.../dev.jsonl` for score features.
- Calibrated cross-encoder probabilities produced in Notebook 04.

## Outputs

- `models/cardinality/model.pkl` containing the trained classifier.
- `models/cardinality/feature_spec.json` documenting feature engineering.

## Steps

1. Generate per-artifact features: top calibrated probabilities s1–s4 (zero-padded), deltas (s1-s2, s2-s3), score entropy, and evidence_type one-hot vectors.
2. Label each artifact with `min(3, |gold_controls|)` using the processed split file.
3. Train a multi-class classifier (e.g., logistic regression or gradient boosting) on train artifacts.
4. Evaluate on dev artifacts, capture accuracy, confusion matrix, and calibration sanity checks.
5. Persist the fitted model and feature spec JSON for reproducible inference.

## Acceptance Checks

- Reported dev accuracy and confusion matrix summarize performance.
- `models/cardinality/model.pkl` and `feature_spec.json` are written.